<a href="https://colab.research.google.com/github/miter37/bert_ner_korean/blob/main/bertner_korean_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
'''
참고한 사이트
#### 케라스 BERT 관련 예제 : https://keras.io/examples/nlp/semantic_similarity_with_bert/
#### BERT 입력값 관련 전처리 : https://androidkt.com/name-entity-recognition-with-bert-in-tensorflow/
#### 한글 NER 데이터 및 국내 KoBERT + CRF 참고 : https://github.com/eagle705/pytorch-bert-crf-ner 
#### LSTM 적용 : https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb
#### Keras BERT 적용 라이브러리 : https://huggingface.co/transformers/model_doc/bert.html?highlight=tfbertmodel#tfbertmodel
#### KoBERT huggingface Transformer 적용 : https://github.com/monologg/KoBERT-Transformers

'''
!nvidia-smi -L
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import tensorflow as tf

print("현재 TF버젼: ",tf.version.VERSION)
print("현재 Keras버젼: ", tf.keras.__version__)
print("Colab Pro, GPU Tesla V100, TF 2.3, Keras 2.4 사용함")

## 경로 지정 : 파일들 설치한 경로를 써주면 됨.
path = '/content/mnt/My Drive/Colab Notebooks/project-2nd seme/BERTNER_2020팀프로젝트/version3'
#path = '/content/mnt/Shareddrives/AI-EDU/알고리즘/BERTNER'
'''구글 드라이브 마운팅'''
import os, sys 
from google.colab import drive 
drive.mount('/content/mnt') 

!pip install transformers==3.0.0
!pip install seqeval
!pip install sklearn_crfsuite
#!pip3 install kobert-transformers
#!pip install git+https://www.github.com/keras-team/keras-contrib.git

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-3404febe-8d81-6fda-16b0-435216d8531e)
현재 TF버젼:  2.3.0
현재 Keras버젼:  2.4.0
Colab Pro, GPU Tesla V100, TF 2.3, Keras 2.4 사용함
Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [ ]:
'''주요 라이브러리 임포트'''
#%tensorflow_version 2.x
#import tensorflow as tf
#print(tf.__version__)

import pandas as pd
import math
import numpy as np

from transformers import ( TF2_WEIGHTS_NAME, BertConfig,
    BertTokenizer, BertModel, TFBertModel, create_optimizer)
from kobert_transformers import get_kobert_model, get_distilkobert_model

from seqeval.metrics import classification_report,accuracy_score,f1_score
from tqdm import tqdm,trange


'''뒤에 발생한 의문의 멈춤현상을 잡기위한 코드'''
import logging
logging.basicConfig(level=logging.ERROR)  #https://github.com/huggingface/transformers/issues/5505#

#데이터 세팅

In [ ]:
''' (시간오래걸림) 한글 ner 데이터 text파일을 하나로 합치는 과정. 한번 했으면 실행할 필요 없음 '''
## concatenate_texts 파일 있으면 실행하지 않아도 됨
'''
## https://hoood.tistory.com/300
cd /content/mnt/My Drive/Colab Notebooks/project-2nd seme/NER_kaggle예제/korean_ner_label
import glob

path = '/content/mnt/My Drive/Colab Notebooks/project-2nd seme/NER_kaggle예제/korean_ner_label'
os.chdir(path)

if os.path.exists("concatenate_texts.txt"):
    os.rename("concatenate_texts.txt", "concatenate_texts_original.txt")
else:
    print("The file does not exist")

read_files = glob.glob("*.txt")

print(read_files)

with open("concatenate_texts.txt", "wb") as outfile:
    for f in read_files:
        i = 0
        line = "***********" + f + "***********" + "\n\n"
        i += 1
        outfile.write(line.encode('utf-8'))
        with open(f, "rb") as infile:
            outfile.write(infile.read())'''

'\n## https://hoood.tistory.com/300\ncd /content/mnt/My Drive/Colab Notebooks/project-2nd seme/NER_kaggle예제/korean_ner_label\nimport glob\n\npath = \'/content/mnt/My Drive/Colab Notebooks/project-2nd seme/NER_kaggle예제/korean_ner_label\'\nos.chdir(path)\n\nif os.path.exists("concatenate_texts.txt"):\n    os.rename("concatenate_texts.txt", "concatenate_texts_original.txt")\nelse:\n    print("The file does not exist")\n\nread_files = glob.glob("*.txt")\n\nprint(read_files)\n\nwith open("concatenate_texts.txt", "wb") as outfile:\n    for f in read_files:\n        i = 0\n        line = "***********" + f + "***********" + "\n\n"\n        i += 1\n        outfile.write(line.encode(\'utf-8\'))\n        with open(f, "rb") as infile:\n            outfile.write(infile.read())'

In [ ]:
''' 한글 ner 학습데이터가 합쳐진 concatenate_texts 불러와서 정리 '''

file_full_name=path + '/concatenate_text.txt'
with open(file_full_name, 'r') as textfile:
  kor_ner_dataset = []
  sen_num = 1
  for row in textfile:
    if row[0:3] != "***" :
      if row[0] != "#": 
        if row[0] != "_":
          if row == "\n":
            sen_num += 1
          else :
            row2 = row.replace("\n","")
            row3 = row2.split("\t")
            row3.insert(0,"sen_"+str(sen_num))
            #print(row3)
            kor_ner_dataset.append(row3)


## 읽은 txt를 판다스 데이터프레임으로 변환하고 컬럼명 지정해줌
df_data = pd.DataFrame(kor_ner_dataset)
df_data.columns = ['Sentence #', 'Word', "Word_2", 'POS', 'Tag']
print(df_data.head())
print(df_data.shape)

## n/a 값 정리
print(df_data.isnull().sum())
df_data = df_data.dropna()
print(df_data.isnull().sum())
print(df_data.shape)

  Sentence # Word Word_2  POS    Tag
0      sen_2    강      강  NNP  B-PER
1      sen_2    씨      씨  NNB      O
2      sen_2    는      는   JX      O
3      sen_2    “      “   SS      O
4      sen_2   처음     처음  NNG      O
(725485, 5)
Sentence #    0
Word          0
Word_2        1
POS           1
Tag           1
dtype: int64
Sentence #    0
Word          0
Word_2        0
POS           0
Tag           0
dtype: int64
(725484, 5)


In [ ]:
## tag의 종류 뽑아내기
tag_list=df_data.Tag.unique()
# ascending=False 하면 내림차순
tag_list =  np.sort(tag_list)[::-1]

print(tag_list) 
print(len(tag_list))

label_map = {label: i+1 for i, label in enumerate(tag_list)}   ##tag_list에 레이블 
idx2label = {i: w for w, i in label_map.items()}  ##tag_list을 뒤집어 놓은거
idx2label[0] = 'Null'
num_labels = len(tag_list) +1

print(label_map)
print(idx2label)

idx2label_k = {1: 'O', 2: 'Time', 3: 'Etc', 4: 'Ratio', 5: 'Name', 6: 'Org', 7: 'Etc_No.', 8: 'Money', 9: 'Location', 10: 'Duration', 11: 'Date', 12: 'Time', 13: 'Etc', 14: 'Ratio',
               15: 'Name', 16: 'Org', 17: 'Etc_No', 18: 'Money', 19: 'Location', 20: 'Duration', 21: 'Date', 0: 'Null'}
print(idx2label_k)
#np.save(path +'/idx2label_k',idx2label_k)

['O' 'I-TIM' 'I-POH' 'I-PNT' 'I-PER' 'I-ORG' 'I-NOH' 'I-MNY' 'I-LOC'
 'I-DUR' 'I-DAT' 'B-TIM' 'B-POH' 'B-PNT' 'B-PER' 'B-ORG' 'B-NOH' 'B-MNY'
 'B-LOC' 'B-DUR' 'B-DAT']
21
{'O': 1, 'I-TIM': 2, 'I-POH': 3, 'I-PNT': 4, 'I-PER': 5, 'I-ORG': 6, 'I-NOH': 7, 'I-MNY': 8, 'I-LOC': 9, 'I-DUR': 10, 'I-DAT': 11, 'B-TIM': 12, 'B-POH': 13, 'B-PNT': 14, 'B-PER': 15, 'B-ORG': 16, 'B-NOH': 17, 'B-MNY': 18, 'B-LOC': 19, 'B-DUR': 20, 'B-DAT': 21}
{1: 'O', 2: 'I-TIM', 3: 'I-POH', 4: 'I-PNT', 5: 'I-PER', 6: 'I-ORG', 7: 'I-NOH', 8: 'I-MNY', 9: 'I-LOC', 10: 'I-DUR', 11: 'I-DAT', 12: 'B-TIM', 13: 'B-POH', 14: 'B-PNT', 15: 'B-PER', 16: 'B-ORG', 17: 'B-NOH', 18: 'B-MNY', 19: 'B-LOC', 20: 'B-DUR', 21: 'B-DAT', 0: 'Null'}
{1: 'O', 2: 'Time', 3: 'Etc', 4: 'Ratio', 5: 'Name', 6: 'Org', 7: 'Etc_No.', 8: 'Money', 9: 'Location', 10: 'Duration', 11: 'Date', 12: 'Time', 13: 'Etc', 14: 'Ratio', 15: 'Name', 16: 'Org', 17: 'Etc_No', 18: 'Money', 19: 'Location', 20: 'Duration', 21: 'Date', 0: 'Null'}


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test=train_test_split(df_data,test_size=0.20,shuffle=False)
x_train.shape,x_test.shape
print(x_train.head())

agg_func = lambda s: [ [w,t] for w,t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]

x_train_grouped = x_train.groupby("Sentence #").apply(agg_func)
x_test_grouped = x_test.groupby("Sentence #").apply(agg_func)
print(x_train_grouped)

x_train_sentences = [[s[0] for s in sent] for sent in x_train_grouped.values]
x_test_sentences = [[s[0] for s in sent] for sent in x_test_grouped.values]
print(x_train_sentences)

x_train_tags = [[t[1] for t in tag] for tag in x_train_grouped.values]
x_test_tags = [[t[1] for t in tag] for tag in x_test_grouped.values]
print(x_train_tags)

print(np.shape(x_train_sentences),np.shape(x_test_sentences))
print(np.shape(x_train_tags),np.shape(x_test_tags))
print(type(x_train_sentences))
print(x_train_sentences[0])

  Sentence # Word Word_2  POS    Tag
0      sen_2    강      강  NNP  B-PER
1      sen_2    씨      씨  NNB      O
2      sen_2    는      는   JX      O
3      sen_2    “      “   SS      O
4      sen_2   처음     처음  NNG      O


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# BERT 모델 세팅

In [ ]:
''' BERT 모델 설정 '''
max_seq_length =128
pad_token=0
pad_token_segment_id=0
sequence_a_segment_id=0
pad_token_label_id = 0
BERT_MODEL="bert-base-multilingual-cased"
config = BertConfig.from_pretrained(BERT_MODEL)     # transformer BERT 라이브러리 사용을 위한 config
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL,do_lower_case=False)    # transformer BERT 라이브러리 중 Tokenizer
#from kobert_transformers import get_tokenizer
#tokenizer = get_tokenizer()


In [ ]:
''' BERT모델의 input에 맞는 형식으로 바꿔주는 함수 '''
from keras.preprocessing.sequence import pad_sequences
from kobert_transformers import get_tokenizer

def convert_to_input(sentences,tags):
  input_id_list,attention_mask_list,token_type_id_list=[],[],[]
  label_id_list=[]
  
  for x,y in tqdm(zip(sentences,tags),total=len(tags)):
  
    tokens = []
    label_ids = []

    for word, label in zip(x, y):
      # 단어를 분절시키는 tokenizer 과정
      word_tokens = tokenizer.tokenize(word)
      tokens.extend(word_tokens)
      # Use the real label id for the first token of the word, and padding ids for the remaining tokens
      if label[0][0] == "B" :
        label_ids.extend([label_map[label]] + [label_map["I"+label[1:]]] * (len(word_tokens) - 1))
      else :
        label_ids.extend([label_map[label]] + [label_map[label]] * (len(word_tokens) - 1))
      ### 원본은 label_ids.extend([label_map[label]] + [0] * (len(word_tokens) - 1))  인데 수정함.      
  
    special_tokens_count =  2
    if len(tokens) > max_seq_length - special_tokens_count:
      tokens = tokens[: (max_seq_length - special_tokens_count)]
      label_ids = label_ids[: (max_seq_length - special_tokens_count)]

    label_ids = [pad_token_label_id]+label_ids+[pad_token_label_id]
    inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length)

    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    attention_masks = [1] * len(input_ids)

    attention_mask_list.append(attention_masks)
    input_id_list.append(input_ids)
    token_type_id_list.append(token_type_ids)

    label_id_list.append(label_ids)

  return input_id_list,token_type_id_list,attention_mask_list,label_id_list

''' 위 함수를 이용해 변환 작업 '''
input_ids_train,token_ids_train,attention_masks_train,label_ids_train=convert_to_input(x_train_sentences,x_train_tags)
input_ids_test,token_ids_test,attention_masks_test,label_ids_test=convert_to_input(x_test_sentences,x_test_tags)

100%|██████████| 4777/4777 [00:05<00:00, 905.70it/s]


In [ ]:
''' 128개 입력값을 받는 BERT 입력값에 맞게 빈칸 채우는 padding작업과, y 결과값을 원핫인코딩 작업 '''

## 패딩, 카테고리컬 직전 출력
kk =0
kk2 = 0
print(np.shape(input_ids_train[kk]),np.shape(token_ids_train[kk]),np.shape(attention_masks_train[kk]),np.shape(label_ids_train[kk]))
print(np.shape(input_ids_train[kk][kk2]),np.shape(token_ids_train[kk][kk2]),np.shape(attention_masks_train[kk][kk2]),np.shape(label_ids_train[kk][kk2]))
print(np.shape(input_ids_train),np.shape(token_ids_train),np.shape(attention_masks_train),np.shape(label_ids_train))
print(input_ids_train[kk])
print(token_ids_train[kk])
print(attention_masks_train[kk])
print(label_ids_train[kk])

## 훈련데이터 패딩 
input_ids_train = pad_sequences(input_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_train = pad_sequences(token_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_train = pad_sequences(attention_masks_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_train = pad_sequences(label_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
## 테스트 데이터 패딩
input_ids_test = pad_sequences(input_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_test = pad_sequences(token_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_test = pad_sequences(attention_masks_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_test = pad_sequences(label_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

## y 결과값 카테고리 원핫인코딩화
from keras.utils import to_categorical
label_ids_train2 = [to_categorical(i, num_classes = num_labels) for i in label_ids_train]
label_ids_test2 = [to_categorical(i, num_classes = num_labels) for i in label_ids_test]

#패딩, 카테고리컬 직후 출력
print(np.shape(input_ids_train[kk]),np.shape(token_ids_train[kk]),np.shape(attention_masks_train[kk]),np.shape(label_ids_train2[kk]))
print(np.shape(input_ids_train[kk][kk2]),np.shape(token_ids_train[kk][kk2]),np.shape(attention_masks_train[kk][kk2]),np.shape(label_ids_train2[kk][kk2]))
print(np.shape(input_ids_train),np.shape(token_ids_train),np.shape(attention_masks_train),np.shape(label_ids_train2))
print(input_ids_train[kk])
print(token_ids_train[kk])
print(attention_masks_train[kk])
print(label_ids_train2[kk])
print(type(input_ids_train))

(26,) (26,) (26,) (26,)
() () () ()
(19188,) (19188,) (19188,) (19188,)
[101, 8924, 118866, 35979, 39671, 23545, 24178, 9637, 9460, 48549, 9043, 9074, 66982, 9328, 18622, 14843, 9460, 9555, 9633, 107657, 9202, 9044, 9557, 9056, 119, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 16, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
(128,) (128,) (128,) (128, 22)
() () () (22,)
(19188, 128) (19188, 128) (19188, 128) (19188, 128, 22)
[   101   8924 118866  35979  39671  23545  24178   9637   9460  48549
   9043   9074  66982   9328  18622  14843   9460   9555   9633 107657
   9202   9044   9557   9056    119    102      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0    

In [ ]:
print(len(label_ids_train2[kk][2]))

# 신경망 모델 작업

## 모델 정의 및 학습

In [ ]:
## Keras model -> https://keras.io/examples/nlp/semantic_similarity_with_bert/
## LSTM, CRF -> https://github.com/Akshayc1/named-entity-recognition/blob/master/NER%20using%20Bidirectional%20LSTM%20-%20CRF%20.ipynb

''' 신경망 모델 작업 현재 모델 model 3 '''

MAX_LENGTH=max_seq_length   ## 위 bert 입력값 만들때 사용한 값
BATCH_SIZE=32   ## 32 이상 코랩에서 안돌아가는 듯

## BERT 레이어 만들기
bert_model = TFBertModel.from_pretrained(BERT_MODEL, config= config)
bert_model.trainable = True     ## BERT 파인튜닝 할 시 True
## BERT 입력값 형식 맞춰주기
input_ids = tf.keras.Input(shape=(MAX_LENGTH,), dtype=tf.int32,name="input_ids"   )
attention_masks = tf.keras.Input(shape=(MAX_LENGTH,),dtype=tf.int32,name="attention_masks"   )
token_type_ids = tf.keras.Input(shape=(MAX_LENGTH,), dtype=tf.int32,name="token_type_ids"  )

## 신경망 레이서 쌓기 시작
sequence_output, pooled_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)

bi_lstm = tf.keras.layers.Bidirectional( tf.keras.layers.LSTM(64*4, return_sequences=True, recurrent_dropout=0.1) )(sequence_output)
#timedist = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64*1, activation = "relu"))(bi_lstm)

#densed = tf.keras.layers.Dense(128*6, activation='relu')(timedist)
#densed = tf.keras.layers.Dense(128*1, activation='relu')(densed)
#dropout = tf.keras.layers.Dropout(0.15)(densed)

#output = tf.keras.layers.Dense(num_labels, activation='softmax')(timedist)    ## output의 마지막 dense는 num_labels와 같아야해
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_labels, activation = "softmax"))(bi_lstm)

### CRF 는 tf.keras에서 현재 지원이 안되어서 포기
#output = crf(timedist)
#crf = CRF(algorithm = 'lbfgs', c1 = 0.1, c2 = 0.1, max_iterations = 100, all_possible_transitions = False)
    # Applying hybrid pooling approach to bi_lstm sequence output.

model = tf.keras.models.Model(  inputs=[input_ids, attention_masks, token_type_ids], outputs=output )

model.compile( optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"] )

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 177853440   input_ids[0][0]                  
                                                                 attention_masks[0][0] 

In [ ]:
## 학습 시작

history = model.fit(
    x = [input_ids_train, token_ids_train, attention_masks_train],
    y = np.array(label_ids_train2),
    validation_data = ([input_ids_test, token_ids_test, attention_masks_test], np.array(label_ids_test2)),
    batch_size = 16*4,
    epochs=15,
    #callbacks=[checkpointer],
    use_multiprocessing=True,
    workers=-1
    )


Epoch 1/15
300/300 [==============================] - 402s 1s/step - loss: 0.0178 - accuracy: 0.9947 - val_loss: 0.0808 - val_accuracy: 0.9814
Epoch 2/15
300/300 [==============================] - 406s 1s/step - loss: 0.0165 - accuracy: 0.9950 - val_loss: 0.0794 - val_accuracy: 0.9819
Epoch 3/15
300/300 [==============================] - 406s 1s/step - loss: 0.0154 - accuracy: 0.9954 - val_loss: 0.0827 - val_accuracy: 0.9813
Epoch 4/15
300/300 [==============================] - 409s 1s/step - loss: 0.0142 - accuracy: 0.9957 - val_loss: 0.0873 - val_accuracy: 0.9807
Epoch 5/15
300/300 [==============================] - 407s 1s/step - loss: 0.0138 - accuracy: 0.9958 - val_loss: 0.0873 - val_accuracy: 0.9808
Epoch 6/15
300/300 [==============================] - 407s 1s/step - loss: 0.0125 - accuracy: 0.9962 - val_loss: 0.0884 - val_accuracy: 0.9812
Epoch 7/15
300/300 [==============================] - 407s 1s/step - loss: 0.0115 - accuracy: 0.9966 - val_loss: 0.0900 - val_accuracy: 0.9812

In [ ]:
'''모델 가중치만 저장. huggingface transformer bert 포함 시 model.save로 하면 에러 발생 '''
#model.save_weights(path + '/ner_model3_2.h5') #epoch 15회
model.save_weights(path + '/ner_model3_3.h5') #epoch 30회

In [ ]:
'''모델 가중치 불러오기 - 기존 학습 모델 가중치'''
model.load_weights(path + '/ner_model3_1.h5')

In [ ]:
# Evaluation
y_pred1 = model.predict([input_ids_test, token_ids_test, attention_masks_test])
y_pred = np.argmax(y_pred1, axis=-1)
y_test_true = np.argmax(label_ids_test2, -1)

In [ ]:
# Convert the index to tag
y_pred2 = [[idx2label_k[i] for i in row] for row in y_pred]
y_test_true2 = [[idx2label_k[i] for i in row] for row in y_test_true]

In [ ]:
from sklearn_crfsuite.metrics import flat_classification_report
report = flat_classification_report(y_pred=y_pred2, y_true=y_test_true2)
print(report)

              precision    recall  f1-score   support

        Date       0.93      0.94      0.94      2875
    Duration       0.68      0.74      0.71      1012
         Etc       0.73      0.70      0.72      7493
      Etc_No       0.87      0.91      0.89      2675
     Etc_No.       0.87      0.89      0.88      3679
    Location       0.74      0.84      0.79      4578
       Money       0.97      0.99      0.98      1132
        Name       0.93      0.94      0.93      9380
        Null       1.00      1.00      1.00    395353
           O       0.98      0.98      0.98    170615
         Org       0.81      0.79      0.80     10991
       Ratio       0.96      0.96      0.96      1176
        Time       0.95      0.89      0.92       497

    accuracy                           0.98    611456
   macro avg       0.88      0.89      0.88    611456
weighted avg       0.98      0.98      0.98    611456



## 결과물 표시에 필요한 함수 정의

In [ ]:
''' 입력된 복수의 문장들을 BERT모델의 input에 맞는 형식으로 리턴하는 함수 '''
from keras.preprocessing.sequence import pad_sequences

def convert_sentences_to_input(sentences):
  input_id_list,attention_mask_list,token_type_id_list=[],[],[]
  label_id_list=[]
  
  for x in sentences:
  
    tokens = []
    label_ids = []

    word_tokens = tokenizer.tokenize(x)
    tokens.extend(word_tokens)
  
    special_tokens_count =  2
    if len(tokens) > max_seq_length - special_tokens_count:
      tokens = tokens[: (max_seq_length - special_tokens_count)]

    inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length)

    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    attention_masks = [1] * len(input_ids)

    attention_mask_list.append(attention_masks)
    input_id_list.append(input_ids)
    token_type_id_list.append(token_type_ids)

  return input_id_list,token_type_id_list,attention_mask_list

## 위 함수를 이용해 변환 작업 
#input_ids_train,token_ids_train,attention_masks_train,label_ids_train=convert_to_input(x_train_sentences,x_train_tags)


In [ ]:
'''# 문장을 넣으면 NER 정보가 포함된 문장으로 바꿔주는 함수 '''

def sentence_ner(sentence, model, cs_input = 0.7) :
  #버트 입력값 만들기
  input_ids_sentence, token_ids_sentence, attention_masks_sentence = convert_sentences_to_input([sentence])
  #버트 형식에 맞게 패딩작업
  input_ids_sentence = pad_sequences(input_ids_sentence,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
  token_ids_sentence = pad_sequences(token_ids_sentence,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
  attention_masks_sentence = pad_sequences(attention_masks_sentence,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
  
  bert_input = [input_ids_sentence, token_ids_sentence,attention_masks_sentence]
  ner_score = model.predict(bert_input)
  ner_output = np.argmax(ner_score, axis=-1)
  
  cs_no = 5
  criteria_score = np.array(ner_score[0][:,2])
  criteria_score.sort()
  cs = criteria_score[-1*cs_no]
  cs = cs_input

  # 문장으로 표시해 주기
  ner_sentence = ''
  for i in range(len(input_ids_sentence[0])) :
    piece = ner_output[0][i]

    if input_ids_sentence[0][i]!=101 and input_ids_sentence[0][i]!=102 and input_ids_sentence[0][i]!=0 : 
      word = tokenizer.decode(input_ids_sentence[0][i:i+1])
      if word[0:2]!="##" : 
        if piece >1.5 and max(ner_score[0][i])>=cs:
          ner_sentence = ner_sentence +' [' + idx2label_k.get(piece) + ']'
        else :
          ner_sentence +=  ' '
          
        if "UNK" in word :
          ner_sentence +=  '"'
        elif "," in word :
          ner_sentence = ner_sentence[:-1]
          ner_sentence +=  ','
        elif "." in word :
          ner_sentence = ner_sentence[:-1]
          ner_sentence +=  '.'
        else :
          ner_sentence +=  word.strip()

      else :
        ner_sentence +=  word[2:]
  return ner_sentence

'\n  # 문장으로 표시해 주기\n  ner_sentence = \'\'\n  for i in range(len(input_ids_sentence[0])) :\n    piece = ner_output[0][i]\n\n    if piece!=0 : \n      word = tokenizer.decode(input_ids_sentence[0][i:i+1])\n      if word[0:2]!="##" : \n        if piece >1.5 :\n          ner_sentence = ner_sentence +\' (\' + idx2label.get(piece) + \')\'\n        else :\n          ner_sentence +=  \' \'\n        ner_sentence +=  word\n      else :\n        ner_sentence +=  word[2:]\n  return ner_sentence\n\n\n  #print(\'\x1b\' + \'다크 오렌지\' + \'\x1b\')\n'

In [ ]:
 ''' 입력문단에 model 돌려 NER정보를  포함해 출력 '''
#!pip install kss
#import kss

def input2ner(sentences, model, cs_no = 0.7) :
  #kss 문장분리기 : 문단을 문장별로 나눠 리스트에 저장해줌
  #senbysen = kss.split_sentences(sentences)
  senbysen = sentences.split('.')
  print("\n\n원문", senbysen,"\n")

  for i in range(len(senbysen)) :
    ner_output = sentence_ner(senbysen[i]+".", model, cs_no)
    print(ner_output)


## NER학습된 모델을 이용 예시 문장의 개체명 분석을 진행하는 부분

In [ ]:
# 고려((나무위키 개요))
input_sentences = "고려(高麗)는 918년부터 1392년까지 약 474년 동안 한반도 지역에 위치했던 전제군주정 불교 국가이다. 통일신라의 분열 후 고려, 후백제, 신라로 나뉘어 대치하던 후삼국시대를 왕건이 936년에 통일하였고, 이후 약 456년 동안 총 34명의 군주가 계승하여 1392년에 이성계의 역성혁명에 의해 조선이 건국되기까지 함경북도와 함경남도 그리고 평안북도의 일부 지역들을 제외한 한반도 대부분의 지역들을 지배하였다. 고려 전기 발해를 멸망시킨 요나라(거란족)와 송나라의 세력 균형 체제라는 국제적인 정세 속에서 고려는 요나라의 3차에 걸친 대규모 침략을 막아내며 자주국방의 위세를 뽐냈으나, 중기 이후 내부적으론 무신정권이 들어서고 외부적으론 금나라(여진족)의 부흥에 요와 북송이 차례로 털리고 금이 고려에 칭신을 요구하자 책봉 질서의 사대적 관계를 맺기 시작했고 이후 몽골 제국과 30년간의 처절한 항쟁 끝에 결국 항복하여 원 간섭기엔 심한 정치적 간섭을 받게 된다. 말이 좋아 항쟁이지 3차 침입 이후부터는 국토의 대부분이 쓸려나가는 동시에 육지의 백성들이 몽골군에게 참혹하게 도륙당하는 그야말로 악몽과도 같은 시기였다. 한편 당시 권력자 집단이었던 무신정권은 강화도에 숨어 호의호식하고 있었다. 다만 금과 남송을 포함한 유라시아 수십여개 국들이 몽골 제국에 쓸려나가는 동안, 고려는 비록 조정은 강화도에서 무능한 모습을 보였지만 백성들의 끈질긴 저항과 적절한 외교술로 나라 자체가 완전히 멸망하진 않았고, 이후 몽골의 부마국으로 존속하다가 고려 후기 원이 쇠퇴하자 공민왕 시절에 반원정책을 펼치면서 몽골의 내정간섭에서 벗어나게 된다." 
# 명량해전 위키
input_sentences2 = "이후 이순신은 붙잡힌 임준영 대신, 준사의 도움으로 풀려난 수봉에게 전갈을 받고 왜군의 전력을 파악한다. 그리고 명량의 흐름도 직접 관측한다. 명량 해전 당일, 이순신은 어머니의 위패에 절을 올리고 수봉에게 참전하고 싶거든 대장선의 노를 저으라 한 뒤 명량으로 올라오는 구루시마 선봉 왜선 330척과 맞선다. 구루시마는 당포 해전에서 죽은 형의 위패를 모신 아타케부네를 타고 적진 진군을 명한다. 이순신은 주저하는 다른 배를 놓아두고 앞장서서 적을 포격한다. 구루시마의 2진이 올 때, 대장선을 더 이상 지탱하기 힘들어지자 이순신은 닻을 끊고 해류 영향이 적은 섬 근처로 배를 옮길 것을 명한다. 그리고는 초요기(장수들을 부르는 명령기)를 올리나 아무도 오지 않는다. 이 때를 놓치지 않고 구루시마는 배를 대장선에 붙일 것을 명령한다. 이후 서너 척의 배에 타고 있던 왜군 병사와 함께 판옥선에서 조선군, 승병, 심지어 노를 젓던 민간인까지 말려든 백병전이 시작된다. 배가 포위되자 이순신은 포를 한 데 모아 터트리라 명령하고, 이는 성공하여 포위에서 풀리게 된다. 때맞춰 안위의 배가 지원을 위해 온다. 하지만 대장선의 화약이 다 떨어진 그 때, 화약과 조선인 포로를 실은 구루시마의 화공선이 대장선을 향해 다가온다. 대장선이 쏜 대장군전 덕에 풀려난 임준영이 아내에게 다른 배가 저 화공선을 보게 해달라는 신호를 보내고, 이를 알게 된 아내가 치마를 벗어 위로 펄럭인다. 구경하던 다른 백성들도 고함을 지르며 옷을 흔들자 이를 알게 된 중군장 김응함의 배가 포탄을 쏘아 대장선까지 닿기 전에 화약선을 터트리는 데 성공한다. 대장선이 멀쩡하다는 데 고무된 다른 배들이 서서히 참전한다. 그러나 구루시마를 제외한 다른 왜선들은 이순신을 두려워하며 전혀 지원하지 않는다. 여기서 이회는 이순신이 말한 '두려움을 이용한다'는 말의 뜻을 알게 된다. 판옥선이 아타케부네를 들이받으며 부수고, 형의 위패가 포격을 맞아 박살나는 걸 본 구루시마는 분노하여 직접 나선다. 이후 회오리 속에 양측의 배가 휘말린다. 구루시마는 직접 대장선으로 올라가 분투하지만 결국 죽고, 자신이 했던 그대로 목이 잘려 깃대에 걸리게 된다. 회오리에 휘말린 대장선은 백성들의 도움으로 겨우 탈출하는 데 성공하고, 도도 다카토라는 와키자카의 배가 피격당하는 것을 보고 난 뒤, '대도무문'이 적힌 깃발을 뒤로 한 채 퇴각을 명한다." 
# 국사교과서 - 갑오개혁
input_sentences3 = "일본의 침략과 급진적인 갑오·을미개혁의 실시로 대부분의 국민 사이에 반일 정서가 확산되었다. 또, 고종은 왕권을 제약하려는 개화 세력의 개혁에 불만을 가지고 있었고, 을미사변 후에는 신변의 위협까지 느끼게 되었다. 이에, 고종은 러시아 공사관으로 피신하였고(아관 파천, 1896), 개화파 정부는 무너졌다. 이후 고종은 단발령 철회, 의병 해산 권고 조치 등을 단행하였다.   아관 파천으로 국가의 자주성은 손상되었고, 광산, 산림 등에 대한 열강의 이권 침탈도 심해졌다. 이러한 상황에서 서재필 등은 독립 신문을 창간하여 서구의 자유 민권 사상을 소개하였으며, 독립 협회를 창립하였다(1896).   독립 협회는 강연회와 토론회 등을 통하여 민중에게 근대적 지식과 국권·민권 사상을 고취시켜, 광범한 사회 계층의 지지를 받는 단체로 발전하였다. 또, 독립 협회는 자주 국권, 자유 민권 등을 달성하려는 정치 운동을 전개하였으며, 만민 공동회와 관민 공동회를 개최하여 헌의 6조를 결의하였다.  그런데 독립 협회의 활동은 의회의 설립과 서구식 입헌 군주제 실현을 목표로 하였기 때문에 보수 세력과 대립하였다. 독립 협회는 보수 세력이 동원한 황국 협회의 방해를 받았고, 결국 3년 만에 해산되고 말았다.   한편, 고종은 러시아 공사관에서 약 1년 만에 환궁한 후, 자주 독립의 근대 국가를 세우려는 국민적 열망과 러시아를 견제하려는 국제 여론에 힘입어 대한 제국을 수립하였다(1897). 고종은 황제로 즉위하면서 연호를 광무(光武)로 하고, 자주 국가임을 국내외에 선포하였다.   대한 제국은 “옛 제도를 근본으로 하고 새로운 제도를 참작한다.”라는 구본신참(舊本新參)의 개혁 방향을 제시하고, 대한국 국제를 제정하여 황권을 강화하였다. 또, 양전 사업을 실시하여 지계를 발급하고, 상공업 진흥책을 추진하였다. 그러나 이러한 개혁 정책은 집권층의 보수적 성향과 열강의 간섭으로 큰 성과를 거두지는 못하였다. "
# 국사교과서 - 조선초기 외교관계
input_sentences4 = "명과 관계   조선은 사대 교린의 외교 정책을 일관되게 추진하였다. 명과는 태조 때 정도전이 중심이 되어 추진한 요동 정벌의 준비와 여진과의 관계를 둘러싸고 불편한 관계가 유지된 적도 있었지만, 태종 이후 양국 간의 관계가 좋아지면서 교류가 활발하였다.   조선은 명에 대해서 기본적으로 사대 정책을 유지하였으나, 명의 구체적인 내정 간섭은 없었다. 매년 정기적, 부정기적으로 사절을 교환하였고, 그 때 문화적, 경제적 교류가 활발하게 이루어졌다. 명에 대한 이와 같은 사대 외교는 왕권의 안정과 국제적 지위를 확보하려는 자주적인 실리 외교였고, 선진 문물을 흡수하려는 문화 외교인 동시에 일종의 공무역이었다.  여진과 관계   조선은 영토의 확보와 국경 지방의 안정을 위하여 여진에 대하여 적극적인 외교 정책을 펴 나갔다. 우선 태조에 의하여 일찍부터 두만강 지역이 개척되었다. 이어 세종 때에는 4군과 6진을 설치하여 압록강과 두만강을 경계로 하는, 오늘날과 같은 국경선을 확정하였다.   이후 여진에 대하여 조선은 회유와 토벌의 양면 정책을 취하였다. 조선은 여진족의 귀순을 장려하기 위하여 관직을 주거나 정착을 위한 토지와 주택을 주어 우리 주민으로 동화시켰다. 또, 사절의 왕래를 통한 무역을 허용하였고, 국경 지방인 경성과 경원에 무역소를 두고 국경 무역을 허락하였다. 그러나 이러한 교린 정책에도 불구하고 여진족은 자주 국경을 침입하여 약탈을 자행하였고, 이 때마다 조선에서는 군대를 동원하여 이들을 정벌하였다.  이와 함께 조선은 삼남 지방의 일부 주민을 대거 북방으로 이주시켜 압록강과 두만강 이남 지역을 개발하는 사민 정책을 실시하였고, 토착민을 토관으로 임명하여 민심을 수습하려 하였다."

In [ ]:
# NER을 문장에 표시해주는 함수 실행
input2ner(input_sentences, model)
input2ner(input_sentences2, model)
input2ner(input_sentences3, model)
input2ner(input_sentences4, model, 0.9)  # 마지막에 optional한 숫자는 표시할 최소 정확도 값을 의미. 0.9를 넣으면 결과값 0.9 이상인 NER만 표시. 기본값 0.7



원문 ['고려(高麗)는 918년부터 1392년까지 약 474년 동안 한반도 지역에 위치했던 전제군주정 불교 국가이다', ' 통일신라의 분열 후 고려, 후백제, 신라로 나뉘어 대치하던 후삼국시대를 왕건이 936년에 통일하였고, 이후 약 456년 동안 총 34명의 군주가 계승하여 1392년에 이성계의 역성혁명에 의해 조선이 건국되기까지 함경북도와 함경남도 그리고 평안북도의 일부 지역들을 제외한 한반도 대부분의 지역들을 지배하였다', ' 고려 전기 발해를 멸망시킨 요나라(거란족)와 송나라의 세력 균형 체제라는 국제적인 정세 속에서 고려는 요나라의 3차에 걸친 대규모 침략을 막아내며 자주국방의 위세를 뽐냈으나, 중기 이후 내부적으론 무신정권이 들어서고 외부적으론 금나라(여진족)의 부흥에 요와 북송이 차례로 털리고 금이 고려에 칭신을 요구하자 책봉 질서의 사대적 관계를 맺기 시작했고 이후 몽골 제국과 30년간의 처절한 항쟁 끝에 결국 항복하여 원 간섭기엔 심한 정치적 간섭을 받게 된다', ' 말이 좋아 항쟁이지 3차 침입 이후부터는 국토의 대부분이 쓸려나가는 동시에 육지의 백성들이 몽골군에게 참혹하게 도륙당하는 그야말로 악몽과도 같은 시기였다', ' 한편 당시 권력자 집단이었던 무신정권은 강화도에 숨어 호의호식하고 있었다', ' 다만 금과 남송을 포함한 유라시아 수십여개 국들이 몽골 제국에 쓸려나가는 동안, 고려는 비록 조정은 강화도에서 무능한 모습을 보였지만 백성들의 끈질긴 저항과 적절한 외교술로 나라 자체가 완전히 멸망하진 않았고, 이후 몽골의 부마국으로 존속하다가 고려 후기 원이 쇠퇴하자 공민왕 시절에 반원정책을 펼치면서 몽골의 내정간섭에서 벗어나게 된다', ''] 

 [Org]고려 ( [Org]高 [Org]麗 ) 는 [Duration]918년부터 [Duration]1392년까지 약 [Duration]474년 동안 [Location]한반도 지역에 위치했던 전제군주정 [Org]불교 국가이다.
 [Org]통일신라의 분열 후 [Org]고려, [Org]후백제, [O